In [2]:
!pip install -r requirements.txt 

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 908.5 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 960.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 980.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 999.2 kB/s eta 0:00:00 eta 0:00:01m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 994.2 kB/s eta 0:00:00 eta 0:00:010:00:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 971.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 953.3 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Basic pylance operations and inter-conversion between formats

In [3]:
import lance

import pandas as pd
import pyarrow as pa

# pandas -> pyarrow -> parquet
df = pd.DataFrame({"a": [5], "b": [10]})
tbl = pa.Table.from_pandas(df)

uri = "/tmp/test.parquet"
pa.dataset.write_dataset(tbl, uri, format='parquet', existing_data_behavior='overwrite_or_ignore')

# parquet -> lance
uri2 = "/tmp/test.lance"
parquet = pa.dataset.dataset(uri, format='parquet')
lance.write_dataset(parquet, uri2, mode='overwrite')
dataset = lance.dataset(uri2)
assert isinstance(dataset, pa.dataset.Dataset)

# lance -> pandas
dataset.to_table().to_pandas()

# lance -> duckdb
import duckdb
duckdb.query("SELECT * FROM dataset LIMIT 10").to_df()

,a,b
0,5,10


In [ ]:
import lance
from lance.vector import vec_to_table
import numpy as np
import struct

nvecs = 1000000
ndims = 128
with open("sift/sift_base.fvecs", mode="rb") as fobj:
    buf = fobj.read()
    data = np.array(struct.unpack("<128000000f", buf[4 : 4 + 4 * nvecs * ndims])).reshape((nvecs, ndims))
    dd = dict(zip(range(nvecs), data))

table = vec_to_table(dd)

uri = "vec_data.lance"
sift1m = lance.write_dataset(table, uri, max_rows_per_group=8192, max_rows_per_file=1024*1024, mode='overwrite', version="2.1")

# print("Applying Product Quantization (PQ)...")
# dataset = lance.dataset(uri)
# dataset.create_index(
#     column="vector",
#     index_type="IVF_PQ",
#     num_partitions=256,
#     num_sub_vectors=16,
#     metric_type="L2",
#     replace=True
# )

TypeError: write_dataset() got an unexpected keyword argument 'version'